In [1]:
import os 
from dotenv import load_dotenv
import pinecone
import numpy as np
from nomic import atlas
import nomic
from tqdm import tqdm 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import DirectoryLoader
import json 

load_dotenv()
openai_key = os.getenv('OPENAI_KEY')
os.environ['OPENAI_API_KEY'] = openai_key

pinecone_api_key = os.getenv('PINECONE_KEY')
pinecone_env_name = os.getenv('PINECONE_ENV')
pinecone_index_name = os.getenv('PINECONE_INDEX')
nomic_api_key = os.getenv('NOMIC_KEY')

pinecone_config = {
    "api_key":pinecone_api_key,
    "env_name":pinecone_env_name,
    "index_name":pinecone_index_name
}


/home/rc/.local/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Vectors Embedding

<font size=4 color=blue> This time I get the embedding vectors locally rather than fetching the vectors from Pinecone, because cloud vector database has limitations of fetching the vectors (bandwidth and keys encryption). For visulization and experiment purpose, it is suggested that embeding and saving the vectors before upserting to vector database

In [2]:
# pdf_directory = "../docs"
# my_loader = DirectoryLoader(pdf_directory, glob='**/*.pdf')
# documents = my_loader.load()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
# docs = text_splitter.split_documents(documents)


In [3]:
# #### Uncomment for first time uses
# embeddings = OpenAIEmbeddings()
# chunk_dicts = {
#     "text":[],
#     "vectors":[],
#     "categories":[]
# }

# for chunk in tqdm(docs):
#     chunk_text = chunk.page_content
#     chunk_embedded_vector = embeddings.embed_documents([chunk_text])    

#     chunk_dicts["vectors"].append(chunk_embedded_vector)
#     chunk_dicts["categories"].append("pdf_chunk_embeddings")
#     chunk_dicts["text"].append([chunk_text])
    

In [4]:
# 
# with open('pdf_embedding.json', 'w') as f:
#     # Use json.dump to write the data to the file.
#     json.dump(chunk_dicts, f)

In [5]:
with open('pdf_embedding.json', 'r') as f:
    # Use json.load to load the data from the file.
    pdf_embedding = json.load(f)

In [6]:
nomic.login(nomic_api_key)
id_list = range(len(pdf_embedding['text']))
project = atlas.map_embeddings(embeddings= np.squeeze(np.array(pdf_embedding["vectors"]),  axis=1),
                               data=[{'id':idx, 'text':text[0], 'category':cat} for idx, text, cat in zip(id_list, pdf_embedding['text'], pdf_embedding['categories'])], 
                               id_field='id',
                               description='pdfs')

2023-07-03 16:41:53.290 | INFO     | nomic.project:_create_project:749 - Creating project `oafish-oxford` in organization `rancheng0918`
2023-07-03 16:41:55.340 | INFO     | nomic.atlas:map_embeddings:107 - Uploading embeddings to Atlas.
2023-07-03 16:41:55.384 | WARNING  | nomic.project:_validate_and_correct_arrow_upload:254 - id_field is not a string. Converting to string from int32
2it [00:03,  1.60s/it]                                                          
2023-07-03 16:41:58.588 | INFO     | nomic.project:_add_data:1371 - Upload succeeded.
2023-07-03 16:41:58.588 | INFO     | nomic.atlas:map_embeddings:126 - Embedding upload succeeded.
2023-07-03 16:42:00.056 | INFO     | nomic.project:create_index:1081 - Created map `oafish-oxford` in project `oafish-oxford`: https://atlas.nomic.ai/map/3d6e6811-f1f1-479c-aae0-8042d9d4542b/8f94a78f-435d-4ffb-bba3-84c2819fd542
2023-07-03 16:42:00.057 | INFO     | nomic.atlas:map_embeddings:139 - oafish-oxford: https://atlas.nomic.ai/map/3d6e681

# Graph Embeddings

In [7]:
from neo4j import GraphDatabase
neo4j_url = os.getenv('NEO4J_URL')
neo4j_user = os.getenv('NEO4J_USER')
neo4j_password = os.getenv('NEO4J_PASSWORD')

In [8]:
# driver = GraphDatabase.driver(neo4j_url, auth=(neo4j_user, neo4j_password))
# # Divide the graph into trunks
# def get_pair_nodes (tx):
#     pairs_of_nodes = []
#     for record in tx.run("MATCH (a)-[r]->(b) RETURN labels(a), a.name, type(r), labels(b), b.name"):
#         pair_node = {
#             "start_node_name": record["a.name"],
#             "start_node_label":record["labels(a)"][0],
#             "end_node_name": record["b.name"],
#             "end_node_label":record["labels(b)"][0],
#             "edge": record["type(r)"]
#         }
#         pairs_of_nodes.append(pair_node)
#     return pairs_of_nodes

# with driver.session() as session:
#     pairs_of_nodes = session.execute_read(get_pair_nodes)
    
# driver.close()

# node_names = set()
# node_types = set()
# edge_types = set()
# for p_n in pairs_of_nodes:
#     node_names.add(p_n["start_node_name"])
#     node_names.add(p_n["end_node_name"])
    
#     node_types.add(p_n["start_node_label"])
#     node_types.add(p_n["end_node_label"])
#     edge_types.add(p_n["edge"])

In [9]:
# node_dicts = {
#     "text":[],
#     "vectors":[],
#     "categories":[]
    
# }

# for n in node_names:
#     node_dicts["text"].append(n)
#     node_dicts["vectors"].append(embeddings.embed_documents([n]))
#     node_dicts["categories"].append("node_name (graphs)")
    
# for n in node_types:
#     node_dicts["text"].append(n)
#     node_dicts["vectors"].append(embeddings.embed_documents([n]))
#     node_dicts["categories"].append("node_type (graphs)")
    
# for e in edge_types:
#     node_dicts["text"].append(e)
#     node_dicts["vectors"].append(embeddings.embed_documents([e]))
#     node_dicts["categories"].append("edge_type (graphs)")
    
# with open('graph_embedding.json', 'w') as f:
#     # Use json.dump to write the data to the file.
#     json.dump(node_dicts, f)  
    

In [10]:
with open('graph_embedding.json', 'r') as f:
    # Use json.load to load the data from the file.
    graph_embedding = json.load(f)

In [11]:
nomic.login(nomic_api_key)
id_list = range(len(graph_embedding['text']))
project = atlas.map_embeddings(embeddings= np.squeeze(np.array(graph_embedding["vectors"]),  axis=1),
                               data=[{'id':idx, 'text':text, 'category':cat} for idx, text, cat in zip(id_list, graph_embedding['text'], graph_embedding['categories'])], 
                               id_field='id')

2023-07-03 16:42:03.614 | INFO     | nomic.project:_create_project:749 - Creating project `sad-fairy` in organization `rancheng0918`
2023-07-03 16:42:05.995 | INFO     | nomic.atlas:map_embeddings:107 - Uploading embeddings to Atlas.
2023-07-03 16:42:05.997 | WARNING  | nomic.project:_validate_and_correct_arrow_upload:254 - id_field is not a string. Converting to string from int32
1it [00:02,  2.71s/it]
2023-07-03 16:42:08.704 | INFO     | nomic.project:_add_data:1371 - Upload succeeded.
2023-07-03 16:42:08.704 | INFO     | nomic.atlas:map_embeddings:126 - Embedding upload succeeded.
2023-07-03 16:42:10.335 | INFO     | nomic.project:create_index:1081 - Created map `sad-fairy` in project `sad-fairy`: https://atlas.nomic.ai/map/cb444d6d-c899-4209-ae69-6d18653d4504/1f53bcd8-effa-4820-89c9-7933394b4da9
2023-07-03 16:42:10.335 | INFO     | nomic.atlas:map_embeddings:139 - sad-fairy: https://atlas.nomic.ai/map/cb444d6d-c899-4209-ae69-6d18653d4504/1f53bcd8-effa-4820-89c9-7933394b4da9


# View PDF emedding and Graph embedding in the same plot

In [12]:
# stack pdf and graph embeddings
pdf_embedding_vectors = np.squeeze(np.array(pdf_embedding["vectors"]),  axis=1)
graph_embedding_vectors = np.squeeze(np.array(graph_embedding["vectors"]),  axis=1)

all_embeddings_vectors = np.vstack((pdf_embedding_vectors,graph_embedding_vectors))
all_embeddings_vectors.shape

(2024, 1536)

In [13]:
# stack data
id_list = range(len(pdf_embedding['text']))

pdf_data = [{'id':idx, 'text':text[0], 'category':cat} for idx, text, cat in zip(id_list, pdf_embedding['text'], pdf_embedding['categories'])]

id_list = range(len(pdf_embedding['text']), len(pdf_embedding['text'])+ len(graph_embedding['text']))
graph_data = [{'id':idx, 'text':text, 'category':cat} for idx, text, cat in zip(id_list, graph_embedding['text'], graph_embedding['categories'])]

all_data = pdf_data + graph_data

In [14]:
project = atlas.map_embeddings(embeddings= all_embeddings_vectors,
                               data=all_data, 
                               id_field='id',
                               colorable_fields=['category']
                              )

2023-07-03 16:42:12.823 | INFO     | nomic.project:_create_project:749 - Creating project `delicious-booking` in organization `rancheng0918`
2023-07-03 16:42:14.689 | INFO     | nomic.atlas:map_embeddings:107 - Uploading embeddings to Atlas.
2023-07-03 16:42:14.707 | WARNING  | nomic.project:_validate_and_correct_arrow_upload:254 - id_field is not a string. Converting to string from int32
2it [00:03,  1.94s/it]                                                          
2023-07-03 16:42:18.582 | INFO     | nomic.project:_add_data:1371 - Upload succeeded.
2023-07-03 16:42:18.583 | INFO     | nomic.atlas:map_embeddings:126 - Embedding upload succeeded.
2023-07-03 16:42:19.871 | INFO     | nomic.project:create_index:1081 - Created map `delicious-booking` in project `delicious-booking`: https://atlas.nomic.ai/map/b8455529-dd3b-4075-85fa-b5dc9cac1f42/306c9ecc-e8cd-4591-b7cc-a5d86a60397a
2023-07-03 16:42:19.872 | INFO     | nomic.atlas:map_embeddings:139 - delicious-booking: https://atlas.nomi